# Challenge on stable compounds: Binary classifier and compound dependent feature engineering
__Author: Dario Rocca__

The labels to be predicted are given in the form [1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0] corresponding to pairs of elements formulaA and formulaB. Specifically, the labels correspond to the 1D binary phase diagram: [100% A, (90% A-10% B), (80% A-20% B), (70% A-30% B), (60% A-40% B), (50% A-50% B), (40% A-60% B), (30% A-70% B), (20% A-80% B), (10% A-90% B), 100% B] where a 1 indicates stability and a 0 indicates instability of the corresponding compound.
In the following the first and last labels will be dropped, as they correspond to the pure elements.

While the problem described above can be straightforwardly adressed by a multilabel classifier, in this notebook I will show how it can be reformulated as a binary classifier. Specifically, we can unroll the vector of labels (for example [0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0]) into a column; now each row, namely each training example, has a single label that can take values 0 or 1. In this way I will get a set of training data that is much larger than the original one, with 2572*9 (n of original training data*n of labels) training examples. Also within this reformulation the problem can be addressed as a binary classification. However, there is a problem: How can I create compound (stoichiometry) specific features? For example, now the (70% A-30% B) compound becomes an "independent" training example and ideally should have at least some specific descriptors to distinguish it from the (60% A-40% B) compound in the same stability vector. Using my intuition I designed features such as differences |featureA-featureB| and weighted averages 0.1*featureA+0.9*featureB for the (10% A-90% B) compound. Then I discovered that there was a rich literature in this field. For details and references see the section Feature Engineering below.  

__The reformulation of the problem described in the notebook has several advantages__: <br> 
- We have more training examples; to be established if they really contain more information
- Differently from multilabel classification, there is plenty of algorithms that can be used as binary classifier
- Introducing compound specific features might boost the "accuracy"
- The problem of the swapping of the elements I previously analized is less crucial in this model
- In a way this reformulation feels more natural

__On the other side there are also drawbacks in the binary classifier formulation.__ I will discuss them in the conclusions and they are the reason why I dropped this approach in the final prediction. 

__Important:__ Following Ubaru et al. PRB 95, 214102 (2017) I introduce here some new elemental properties which were missing in the data provided: Cohesive energy, enthalpy of vaporization and electron affinity.

## Loading data

In [2]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Loading the training set with Pandas

train=pd.read_csv("training_data.csv") # creating the train dataframe

#Following Ubaru et al. PRB 95, 214102 (2017)
#I introduce here some new elemental properties which were missing in the data provided
#Cohesive energy, Enthalpy of Vaporization and Electron Affinity.
#Source of the data http://phases.imet-db.ru/elements/main.aspx (I checked some data and it seems reliable)

cohesiveE=pd.read_csv("cohesive_energy.csv")            # Loading cohesive energies for each element
enthalpy_vap=pd.read_csv("enthalpy_vaporization.csv")   # Loading enthalpy of vaporization for each element
elec_affinity=pd.read_csv("electron_affinity.csv")      # Loading electron affinity for each element

# For convenience I'm setting the index of the dataframe to be the atom label
cohesiveE=cohesiveE.set_index('atom')
enthalpy_vap=enthalpy_vap.set_index('atom')
elec_affinity=elec_affinity.set_index('atom')

# For convenience the series is transformed into a dictionary
cohesiveE_dic=cohesiveE['energy'].to_dict()
enthalpy_vap_dic=enthalpy_vap['energy'].to_dict()
elec_affinity_dic=elec_affinity['energy'].to_dict()

# I create the new features columns; at this stage these columns
# contain only the atoms name
train["formulaA_cohesiveE"]=train["formulaA"]
train["formulaB_cohesiveE"]=train["formulaB"]
train["formulaA_enthalpy_vap"]=train["formulaA"]
train["formulaB_enthalpy_vap"]=train["formulaB"]
train["formulaA_elec_affinity"]=train["formulaA"]
train["formulaB_elec_affinity"]=train["formulaB"]

# Using the dictionaries I replace in the previously created columns the
# name of the element with its corresponding property
train=train.replace({"formulaA_cohesiveE": cohesiveE_dic})
train=train.replace({"formulaB_cohesiveE": cohesiveE_dic})
train=train.replace({"formulaA_enthalpy_vap": enthalpy_vap_dic})
train=train.replace({"formulaB_enthalpy_vap": enthalpy_vap_dic})
train=train.replace({"formulaA_elec_affinity": elec_affinity_dic})
train=train.replace({"formulaB_elec_affinity": elec_affinity_dic})

__Below some manipulations that will be useful later.__

In [4]:
# Creating 2 lists that contain the name of the features (columns) corresponding
# to element A and B

columns_A = [] # List that will contain the name of the columns containing the descriptors of element A
columns_B = [] # Same thing for element B

for col in list(train.columns):
    if ('formulaA' in col):
        columns_A.append(col)
    if ('formulaB' in col):
        columns_B.append(col)

# Adding avg_coordination_A (or B) and avg_nearest_neighbor_distance_A (or B) that do not contain
# the string formulaA or formulaB
columns_A.append('avg_coordination_A')
columns_A.append('avg_nearest_neighbor_distance_A')

columns_B.append('avg_coordination_B')
columns_B.append('avg_nearest_neighbor_distance_B')

In [5]:
# The labels to predict are strings of the form "[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0]"
# Below I will extract the corresponding list

import ast

list_label = train['stabilityVec'].tolist()    # A list that contains the column 'stabilityVec'
list_label = map(ast.literal_eval,list_label)
list_label_np = np.asarray(list_label)

#Let's drop the stability index of the pure element, as we do not need to predict it
list_label_compound = list_label_np[:,1:10]

## Creating training and hold out test set

I want to avoid to split the same element pair between taining and hold out test set. For this reason I do it here before feature engineering. 

In [6]:
# Splitting the provided labeled data into training and hold out test set 

# Importing sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_tot = train  
Y_tot = list_label_compound 

# Holding out 10% of the data to test the model 
x_train, x_test, y_train, y_test = train_test_split(X_tot, Y_tot, test_size=0.1, random_state=0)
# x_test corresponds to the hold out set

# Not shown here but I verified a posteriori that this splitting is well stratified both for 
# what concerns the stoichiometric classes and the stable/non-stable labels

## Unrolling the labels

The vector of stability diagrams [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  1.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  1.],.... becomes a single stability vector [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 0.  0.  0.  1.  0.  0.  0.  0.  0. 0.  0.  0.  0.  0.  0.  0.  0.  1. ..............

In [7]:
dim_train = y_train.shape[0]*y_train.shape[1]
Y_train_unroll = reshape(y_train,(dim_train))


dim_test = y_test.shape[0]*y_test.shape[1]
Y_test_unroll = reshape(y_test,(dim_test))

## Feature engineering 

Here I create compound (stoichiometry) specific features; fo the moment I will keep also the original features corresponding to element A and element B.

__Training data__

In [8]:
#These are the original features but each one of them is now
# repeated 9 times (the number of labels)

coltot = list(x_train.columns)

train_dic = {}
for i in coltot:
    col_tmp = list(x_train[i])
    lst = []
    for j in col_tmp:
        lst.extend([j]*9)
    train_dic[i]=lst

#################################################################### 
    
# These are 8 columns containing dummy variables corresponding
# to the different mixture classes (stoichiometries) 90%-10%, 80%-20%, etc. 
# As usually done with one-hot-encoded variables, 8 classes are sufficient to
# describe 9 labels
    
for i in range(8):
    col_name = "mixing_class_"+str(i)
    col_tmp = list(x_train['formulaA'])
    lst = [] 
    lst_tmp=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    lst_tmp[i]=1.0
    for j in range(len(col_tmp)):
        lst.extend(lst_tmp)
    train_dic[col_name]=lst 
    
#################################################################### 

# I'm introducing two columns that contain the actual
# mixing percentage: For example for a 90%-10% mixture
# we will have a column percentage_A with 0.9 and a column
# prcentage_A with 0.1
    
for i in ["A","B"]:
    col_name = "percentage_"+str(i)
    col_tmp = list(x_train['formulaA'])
    lst = [] 
    if i=="A":
        lst_tmp=[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    elif i=="B":
        lst_tmp=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for j in range(len(col_tmp)):
        lst.extend(lst_tmp)
    train_dic[col_name]=lst 

#################################################################### 
    
# The previous data were created as a dictionary that I will trnasform in dataframe    
X_train_unroll = pd.DataFrame(train_dic)    

#################################################################### 

# Features correponding to the L^p norms of the stoichiometry
# These are the stoichiometric attributes in supplementary material of 
# Ward et al. npj Computational Materials 2, 16028 (2016).
# Actually, for this problem these features are not very helpful
for i in range(2,11):
    col_name="norm_stoichiometric_"+str(i)
    X_train_unroll[col_name]=(abs(X_train_unroll["percentage_A"])**i+abs(X_train_unroll["percentage_B"])**i)**(1./i)

#################################################################### 

# I (re)discovered the differences and weighted average features on my own
# However, they were already used together with average deviation
# in Ward et al. npj Computational Materials 2, 16028 (2016).

for i in range(1,len(columns_A)): 
    col_name_diff = "diff_"+columns_A[i]
    col_name_mean = "mean_"+columns_A[i]
    col_name_dev = "dev_"+columns_A[i]
    X_train_unroll[col_name_diff]=abs(X_train_unroll[columns_A[i]]-X_train_unroll[columns_B[i]])
    X_train_unroll[col_name_mean]=(X_train_unroll["percentage_A"]*X_train_unroll[columns_A[i]]
                                   +X_train_unroll["percentage_B"]*X_train_unroll[columns_B[i]])
    X_train_unroll[col_name_dev]=(X_train_unroll["percentage_A"]*
                                  abs(X_train_unroll[columns_A[i]]-X_train_unroll[col_name_mean])
                                   +X_train_unroll["percentage_B"]*
                                  abs(X_train_unroll[columns_B[i]]-X_train_unroll[col_name_mean]))
 
#################################################################### 
 
#The following features correspond to the valence orbital occupation attributes 
#These features have been proposed in Meredig et al. PRB 89 094104 (2014)

X_train_unroll["Fs"]=((X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NsValence"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NsValence"])/
                      (X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NValance"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NValance"]))

X_train_unroll["Fp"]=((X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NpValence"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NpValence"])/
                      (X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NValance"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NValance"]))
                      
X_train_unroll["Fd"]=((X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NdValence"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NdValence"])/
                      (X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NValance"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NValance"]))            
                      
X_train_unroll["Ff"]=((X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NfValence"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NfValence"])/
                      (X_train_unroll["percentage_A"]*X_train_unroll["formulaA_elements_NValance"]
                       +X_train_unroll["percentage_B"]*X_train_unroll["formulaB_elements_NValance"]))

__Hold out test set__ Same as before for the training data

In [9]:
coltot = list(x_test.columns)

test_dic = {}
for i in coltot:
    col_tmp = list(x_test[i])
    lst = []
    for j in col_tmp:
        lst.extend([j]*9)
    test_dic[i]=lst  

#################################################################### 

for i in range(8):
    col_name = "mixing_class_"+str(i)
    col_tmp = list(x_test['formulaA'])
    lst = [] 
    lst_tmp=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    lst_tmp[i]=1.0
    for j in range(len(col_tmp)):
        lst.extend(lst_tmp)
    test_dic[col_name]=lst   
    
#################################################################### 
    
for i in ["A","B"]:
    col_name = "percentage_"+str(i)
    col_tmp = list(x_test['formulaA'])
    lst = [] 
    if i=="A":
        lst_tmp=[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    elif i=="B":
        lst_tmp=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for j in range(len(col_tmp)):
        lst.extend(lst_tmp)
    test_dic[col_name]=lst 
    
#################################################################### 

X_test_unroll = pd.DataFrame(test_dic)    

#################################################################### 

for i in range(2,11):
    col_name="norm_stoichiometric_"+str(i)
    X_test_unroll[col_name]=(abs(X_test_unroll["percentage_A"])**i+abs(X_test_unroll["percentage_B"])**i)**(1./i)

#################################################################### 

for i in range(1,len(columns_A)): 
    col_name_diff = "diff_"+columns_A[i]
    col_name_mean = "mean_"+columns_A[i]
    col_name_dev = "dev_"+columns_A[i]
    X_test_unroll[col_name_diff]=abs(X_test_unroll[columns_A[i]]-X_test_unroll[columns_B[i]])
    X_test_unroll[col_name_mean]=(X_test_unroll["percentage_A"]*X_test_unroll[columns_A[i]]
                                   +X_test_unroll["percentage_B"]*X_test_unroll[columns_B[i]])
    X_test_unroll[col_name_dev]=(X_test_unroll["percentage_A"]*
                                  abs(X_test_unroll[columns_A[i]]-X_test_unroll[col_name_mean])
                                   +X_test_unroll["percentage_B"]*
                                  abs(X_test_unroll[columns_B[i]]-X_test_unroll[col_name_mean]))
    
#################################################################### 
    
X_test_unroll["Fs"]=((X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NsValence"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NsValence"])/
                      (X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NValance"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NValance"]))

X_test_unroll["Fp"]=((X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NpValence"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NpValence"])/
                      (X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NValance"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NValance"]))
                      
X_test_unroll["Fd"]=((X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NdValence"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NdValence"])/
                      (X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NValance"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NValance"]))            
                      
X_test_unroll["Ff"]=((X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NfValence"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NfValence"])/
                      (X_test_unroll["percentage_A"]*X_test_unroll["formulaA_elements_NValance"]
                       +X_test_unroll["percentage_B"]*X_test_unroll["formulaB_elements_NValance"]))

## Feature selection

At this point I have 281 features. To avoid overfitting and spare computer time I included only 100 features selected by the random forest algorithm (not shown). With the exception of the Mendeleev Number all the other features are 
engineered features.

In [10]:
# A list of the features that I will keep
# mixing_class_0, mixing_class_1, mixing_class_2 etc. are not among the most important for the random forest
# classifier but they are necessary to obtain reasonable results with the SVC  
col_to_keep = ["dev_formulaA_elements_NpUnfilled", "dev_formulaA_elements_Column", 
               "dev_formulaA_elements_NdValence", "mixing_class_0", 
               "mixing_class_1", "mixing_class_2", "mixing_class_3", 
               "mixing_class_4", "mixing_class_5", "mixing_class_6", 
               "mixing_class_7", "mean_formulaA_elements_Column", 
               "diff_formulaA_elements_GSenergy_pa", "mean_formulaA_elements_MendeleevNumber", 
               "mean_formulaA_elements_FirstIonizationEnergy", "dev_formulaA_elements_GSenergy_pa", 
               "dev_formulaA_elements_AtomicVolume", "dev_formulaA_elements_HeatCapacityMolar", 
               "mean_formulaA_elements_HHIp", "dev_formulaA_elec_affinity", 
               "dev_formulaA_elements_ElectronSurfaceDensityWS", "mean_formulaA_elements_HeatCapacityMolar", 
               "mean_formulaA_elec_affinity", "mean_formulaA_elements_GSbandgap", 
               "diff_formulaA_elements_HeatCapacityMolar", "mean_formulaA_elements_IsFBlock", 
               "percentage_B", "mean_formulaA_elements_MiracleRadius", 
               "mean_formulaA_elements_Polarizability", "percentage_A", 
               "mean_formulaA_elements_ElectronSurfaceDensityWS", 
               "diff_formulaA_elements_ElectronSurfaceDensityWS", 
               "mean_formulaA_elements_CovalentRadius", "mean_formulaA_elements_Electronegativity", 
               "mean_formulaA_elements_BulkModulus", "dev_formulaA_elements_MendeleevNumber", 
               "mean_formulaA_elements_HeatCapacityMass", "diff_formulaA_elements_AtomicVolume", 
               "dev_avg_nearest_neighbor_distance_A", "dev_formulaA_elements_MeltingT", 
               "mean_formulaA_elements_GSenergy_pa", "mean_formulaA_elements_HHIr", 
               "dev_formulaA_elements_HeatFusion", "mean_formulaA_elements_NUnfilled", 
               "mean_formulaA_elements_Density", "dev_formulaA_elements_Electronegativity", 
               "dev_formulaA_elements_FirstIonizationEnergy", "mean_formulaA_elements_NpUnfilled", 
               "dev_formulaA_elements_HHIr", "Fd", "dev_formulaA_elements_BulkModulus", 
               "dev_formulaA_elements_MiracleRadius", "dev_formulaA_cohesiveE", 
               "dev_formulaA_elements_BoilingT", "mean_formulaA_enthalpy_vap", 
               "mean_formulaA_elements_BoilingT", "dev_formulaA_elements_HHIp", 
               "mean_formulaA_elements_SpaceGroupNumber", "dev_formulaA_enthalpy_vap", 
               "mean_formulaA_elements_HeatFusion", "mean_formulaA_elements_AtomicVolume", "Fs", 
               "mean_formulaA_elements_ShearModulus", "dev_formulaA_elements_Density", 
               "mean_formulaA_cohesiveE", "dev_formulaA_elements_CovalentRadius", 
               "dev_formulaA_elements_HeatCapacityMass", "diff_formulaA_elements_MiracleRadius", 
               "dev_formulaA_elements_SpaceGroupNumber", "diff_formulaA_elements_HeatFusion", 
               "mean_formulaA_elements_MeltingT", "mean_formulaA_elements_NpValence", 
               "mean_avg_nearest_neighbor_distance_A", "diff_formulaA_cohesiveE", 
               "dev_formulaA_elements_GSvolume_pa", "dev_formulaA_elements_Polarizability", 
               "dev_formulaA_elements_GSestBCClatcnt", "dev_formulaA_elements_ShearModulus", 
               "dev_avg_coordination_A", "diff_avg_nearest_neighbor_distance_A", 
               "mean_formulaA_elements_NValance", "mean_formulaA_elements_AtomicWeight", 
               "mean_formulaA_elements_ICSDVolume", "Fp", "diff_formulaA_elec_affinity", 
               "dev_formulaA_elements_AtomicWeight", "mean_formulaA_elements_NdValence", 
               "dev_formulaA_elements_ICSDVolume", "mean_avg_coordination_A", 
               "mean_formulaA_elements_GSestBCClatcnt", "mean_formulaA_elements_Number", 
               "diff_formulaA_elements_MeltingT", "formulaA_elements_MendeleevNumber", 
               "formulaB_elements_MendeleevNumber", "mean_formulaA_elements_GSestFCClatcnt", 
               "diff_formulaA_elements_Column", "dev_formulaA_elements_GSestFCClatcnt", "Ff", 
               "diff_formulaA_elements_NpUnfilled", "mean_formulaA_elements_GSvolume_pa"]

X_train_unroll=X_train_unroll[col_to_keep]
X_test_unroll=X_test_unroll[col_to_keep]

X_train = X_train_unroll 
Y_train = Y_train_unroll 

X_test = X_test_unroll 
Y_test = Y_test_unroll 

## Training and testing a random forest classifier

By using 10-fold cross validation on the training set I performed a grid search for the best parameters for the random forest classifier. I used as scoring function the F1 function. Optimizing this metric leads to a good compromise between precision and recall. This is not exactly like training on subset accuracy (accuracy on the full stability vector) but it's a reasonable choice. The code lines corresponding to the grid search take a lot of time to execute and have been commented. With the optimized parameters I then evaluate the performance on the hold out (test) set.   

In [11]:
# Importing sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import coverage_error 
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import make_scorer

################################################################################

####### The following commented lines have been used to optimize the RandomForestClassifier parameters
####### This takes some time 

## Important parameters to optimize: n_estimators, max_features, and max_depth

# search_grid_rf = [{'n_estimators': [50, 100, 150, 200, 250, 300],
#                  'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
#                  'max_depth': [14, 18, 22, 26, 30]}] 

# clf_rf = GridSearchCV(RandomForestClassifier(random_state=0), search_grid_rf, cv=10,
#                         scoring='f1', verbose=10, n_jobs=4)

# clf_rf.fit(X_train, Y_train)

# print "Best parameters set found on traning set:"
# print ""
# print clf_rf.best_params_
# print ""
# print "Grid scores on training set:"
# print ""

# means = clf_rf.cv_results_['mean_test_score']
# stds = clf_rf.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, clf_rf.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r"
#             % (mean, std * 2, params))
# print ""

# Best parameters set found on traning set:

# {'max_features': 0.5, 'n_estimators': 100, 'max_depth': 26}

# 0.553 (+/-0.073) for {'max_features': 0.5, 'n_estimators': 100, 'max_depth': 26}

################################################################################

# The random forest classifier
clf_rf = RandomForestClassifier(max_features = 0.5, n_estimators = 100, max_depth = 26, random_state=0)
# With the parameters above I got 0.553 (+/-0.073) as best score (F1) in the grid search
# random_state=0 allows for reproducibility; alternatively we could release this constraint 
# and average over different final results

###########################

# Evaluating predictions on training set
clf_rf.fit(X_train, Y_train)
predictions_train = clf_rf.predict(X_train)

print "Detailed classification report:"
print ""
print "The model is trained on the training set."
print("The scores are computed on the training set.")
print ""
print(classification_report(Y_train, predictions_train))
print ""

predictions_train = clf_rf.predict(X_train)

#In order to evaluate the subset accuracy I need to reshape the arrays
lenr = len(Y_train)/9
Y_train_reshape = reshape(Y_train,(lenr,9))
predictions_train_reshape = reshape(predictions_train,(lenr,9))

print "Subset accuracy (accuracy on full stability vector)", accuracy_score(Y_train, predictions_train)

###########################

# Evaluating predictions on hold out test set
predictions_test = clf_rf.predict(X_test)

print "Detailed classification report:"
print ""
print "The model is trained on the training set."
print("The scores are computed on the hold out test set.")
print ""
print(classification_report(Y_test, predictions_test))
print ""

lenr = len(Y_test)/9
Y_test_reshape = reshape(Y_test,(lenr,9))
predictions_test_reshape = reshape(predictions_test,(lenr,9))

print "Subset accuracy (accuracy on full stability vector)", accuracy_score(Y_test_reshape, predictions_test_reshape)

Detailed classification report:

The model is trained on the training set.
The scores are computed on the training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     18590
        1.0       1.00      1.00      1.00      2236

avg / total       1.00      1.00      1.00     20826


Subset accuracy (accuracy on full stability vector) 1.0
Detailed classification report:

The model is trained on the training set.
The scores are computed on the hold out test set.

             precision    recall  f1-score   support

        0.0       0.94      0.99      0.96      2065
        1.0       0.85      0.50      0.63       257

avg / total       0.93      0.93      0.93      2322


Subset accuracy (accuracy on full stability vector) 0.639534883721


__Remarks on random forest classifier__ <br>
- In 10-fold cross validation I obtained 0.553 (+/-0.073) as best F1 score. It is important to notice that the 10 folds can divide the element pairs; for example, the compound (90%A-10%B) can be in one fold and (80%A-20%B) can be in a different one. 
- On the hold out test set I obtain a 0.63 F1 score with a precision that is sizeably higher than the recall. The subset accuracy on the hold out test set is 0.64.  

## Support vector classifier

By using 10-fold cross validation on the training set I performed a grid search for the best parameters for the support vector classifier with a Gaussian kernel. I used as scoring function the F1 function. Optimizing this metric leads to a good compromise between precision and recall. This is not exactly like training on subset accuracy (accuracy on the full stability vector) but it's a reasonable choice. The code lines corresponding to the grid search take a lot of time to execute and have been commented. With the optimized parameters I then evaluate the performance on the hold out (test) set. 

In [12]:
# Importing sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

# Differently from random forests SVC requires feature scaling
# To prevent data leakage I fit the scaler only to X_train

scaler=MaxAbsScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

####### The following commented lines have been used to optimize the SVC parameters
####### This is faster than random forest (only 2 parameters)

## Parameters to optimize: C and gamma
# search_grid_svc = [{'C': [0.1, 1.0, 5.0, 10.0, 20.0, 60.0, 80.0, 100.0, 120.0, 150.0, 200.0, 300.0], 
#                  'gamma': [0.0005, 0.001, 0.002, 0.005, 0.008, 0.01, 0.05, 0.1, 1.0]}] 

# clf_svc = GridSearchCV(SVC(random_state=0), search_grid_svc, cv=10,
#                         scoring='f1', verbose=10, n_jobs=4)

# clf_svc.fit(X_train_scaled, Y_train)

# print "Best parameters set found on traning set:"
# print ""
# print clf_svc.best_params_
# print ""
# print "Grid scores on training set:"
# print ""

# means = clf_svc.cv_results_['mean_test_score']
# stds = clf_svc.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, clf_svc.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r"
#             % (mean, std * 2, params))
# print ""

# Best parameters set found on traning set:

# {'C': 100.0, 'gamma': 0.1}

# 0.622 (+/-0.045) for {'C': 100.0, 'gamma': 0.1}

################################################################################

# The classifier
clf_svc = SVC(C=100.0, gamma=0.1, random_state=0)
# With the parameters above a got 0.622 (+/-0.045) best score in the grid search

# Fitting the classifier
clf_svc.fit(X_train_scaled, Y_train)

predictions_svc = clf_svc.predict(X_test_scaled)

# Printing precision, recall, etc. with respect to the test set 
print "Detailed classification report:"
print ""
print "The model is trained on the training set."
print("The scores are computed on the hold out test set.")
print ""
print(classification_report(Y_test, predictions_svc))
print ""

lenr = len(Y_test)/9
Y_test_reshape = reshape(Y_test,(lenr,9))
predictions_svc_reshape = reshape(predictions_svc,(lenr,9))

print "Subset accuracy (accuracy on full stability vector)", accuracy_score(Y_test_reshape, predictions_svc_reshape)

Detailed classification report:

The model is trained on the training set.
The scores are computed on the hold out test set.

             precision    recall  f1-score   support

        0.0       0.96      0.97      0.96      2065
        1.0       0.72      0.66      0.69       257

avg / total       0.93      0.93      0.93      2322


Subset accuracy (accuracy on full stability vector) 0.635658914729


__Remarks on SVC__ <br>
- In 10-fold cross validation I obtained 0.622 (+/-0.045) as best F1 score. It is important to notice that the 10 folds can divide the element pairs; for example, the compound (90%A-10%B) can be in one fold and (80%A-20%B) can be in a different one. 
- On the hold out test set I obtain a 0.69 F1 score with a precision that is comparable to recall. The subset accuracy on the hold out test set is 0.635, slightly worst than RF. 

## Neural network classifier

I used Keras to create a fully connected neural network model. By using 10-fold cross validation on the training set I performed a grid search for the best parameters for the random forest classifier. As the grid search with 10-fold cross validation is computationally very demanding I limited the number of layers.
I used as scoring function the F1 function. Optimizing this metric leads to a good compromise between precision and recall. This is not exactly like training on subset accuracy (accuracy on the full stability vector) but it's a reasonable choice. The code lines corresponding to the grid search take a lot of time to execute and have been commented. With the optimized parameters I then evaluate the performance on the hold out (test) set. 

In [13]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV

# The model; n_labels and dim have the correct default labels (a bit hard coded)
def nn_model(n_layers=1, n_neuron1=1, n_neuron2=1, drpout=0.0, n_labels=1, dim=100):
    # create nn model
    
    # two layers maximum
    if (n_layers>2):
        return -1
    
    # sequential model
    model = Sequential()
    
    # layer 1
    model.add(Dense(n_neuron1, input_dim=dim, activation='relu'))
    model.add(Dropout(drpout))
    
    # layer 2
    if n_layers == 2:
        model.add(Dense(n_neuron2, activation='relu'))
    model.add(Dropout(drpout))
    
    # output layer 
    model.add(Dense(n_labels, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# To interface Keras and Scikit-learn I need to wrap the Keras model with KerasClassifier
model = KerasClassifier(build_fn=nn_model, epochs=50, batch_size=16, verbose=0)

# Differently from random forests NN require feature scaling
# To prevent data leakage I fit the scaler only to X_train
scaler=MaxAbsScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


####### The following commented lines have been used to optimize the NN parameters
####### Extremely slow
####### I fixed the number of epochs to 50 

#search_grid = [{'n_layers': [1], 
#                'n_neuron1': [20, 60, 100, 150, 200, 250, 300], 
#                'n_neuron2': [20, 60, 100, 150, 200, 250, 300], 
#                'drpout': [0.0, 0.1, 0.2, 0.3, 0.4]}]
#clf_nn = GridSearchCV(estimator=model, param_grid=search_grid, cv=10, verbose=10, n_jobs=4,
#                        scoring='f1')
#clf_nn.fit(X_train_scaled, Y_train)

#print "Best parameters set found on traning set:"
#print ""
#print clf_nn.best_params_
#print ""
#print "Grid scores on training set:"
#print ""

#means = clf_nn.cv_results_['mean_test_score']
#stds = clf_nn.cv_results_['std_test_score']
#for mean, std, params in zip(means, stds, clf_nn.cv_results_['params']):
#    print("%0.3f (+/-%0.03f) for %r"
#            % (mean, std * 2, params))
#print ""


#Best parameters set found on traning set:

#{'drpout': 0.0, 'n_neuron1': 150, 'n_layers': 1}

#0.606 (+/-0.032) for {'drpout': 0.0, 'n_neuron1': 150, 'n_layers': 1}

Using TensorFlow backend.


In [13]:
# In this frame I evaluate the performance of the nn on the hold out test set
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV

#Scaling features
scaler=MaxAbsScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fitting the model
clf_nn = nn_model(n_layers=1, n_neuron1=150, drpout=0.0)
clf_nn.fit(X_train_scaled, Y_train, batch_size=16, epochs=50)

# The nn predicts probabilities
# I fix the threshold to 0.5 to make stability/instability predictions
predictions_nn = clf_nn.predict(X_test_scaled)
predictions_nn[predictions_nn>= 0.5] = 1
predictions_nn[predictions_nn<0.5] = 0


# Printing precision, recall, etc. with respect to the test set 
print "Detailed classification report:"
print ""
print "The model is trained on the training set."
print("The scores are computed on the hold out test set.")
print ""
print(classification_report(Y_test, predictions_nn))
print ""

lenr = len(Y_test)/9
Y_test_reshape = reshape(Y_test,(lenr,9))
predictions_nn_reshape = reshape(predictions_nn,(lenr,9))

print "Subset accuracy (accuracy on full stability vector)", accuracy_score(Y_test_reshape, predictions_nn_reshape)

Epoch 1/50
20826/20826 [==============================] - 2s 99us/step - loss: 0.2600
Epoch 2/50
20826/20826 [==============================] - 2s 81us/step - loss: 0.2330
Epoch 3/50
20826/20826 [==============================] - 2s 81us/step - loss: 0.2215
Epoch 4/50
20826/20826 [==============================] - 2s 82us/step - loss: 0.2118
Epoch 5/50
20826/20826 [==============================] - 2s 84us/step - loss: 0.2061
Epoch 6/50
20826/20826 [==============================] - 2s 87us/step - loss: 0.1997
Epoch 7/50
20826/20826 [==============================] - 2s 85us/step - loss: 0.1953
Epoch 8/50
20826/20826 [==============================] - 2s 82us/step - loss: 0.1921
Epoch 9/50
20826/20826 [==============================] - 2s 81us/step - loss: 0.1883
Epoch 10/50
20826/20826 [==============================] - 2s 89us/step - loss: 0.1862
Epoch 11/50
20826/20826 [==============================] - 2s 89us/step - loss: 0.1831
Epoch 12/50
20826/20826 [===========================

__Remarks on SVC__ <br>
- In 10-fold cross validation I obtained 0.606 (+/-0.032) as best F1 score. It is important to notice that the 10 folds can divide the element pairs; for example, the compound (90%A-10%B) can be in one fold and (80%A-20%B) can be in a different one. 
- On the hold out test set I obtain a 0.65 F1 score with a precision that is comparable to recall. The subset accuracy on the hold out test set is about 0.6, worst than RF and SVC.
- No random seed fixed: The results fluctuate a bit rerunning the code.

## General Conclusions 

I trained RF, SVC, and NN on the binary classification problem. I also rapidly tried the k-nearest neighbours and 
Gaussian naive Bayes classifiers without a sizeable improvement in the results (not shown in the notebook). Overall the performance of all the three algorithms is similar. I guess that this is due to a good work in the feature engineering that "leave less work" to the algorithms. The classifier that seems to perform better is the SVC. <br>

However, I would like to highlight a few concerns about the models in this notebook:
- The models are trained using F1 as metric. While this is a reasonable choice the results in the subset accuracy can have sizeable fluctuations.
- Training these models on subset accuracy (the full stability vector) is both technically difficult and conceptually not well defined. Indeed, now each compound with its own stoichiometry becomes to a certain extent an independent system.  
- In a way this approach might loose the dependence between the labels in the same stability vector. For example, the stability of the (90% A-10% B) compound might have implication on the stability of (80% A-20% B).

In conclusion, I believe that the models in this notebook are more suitable if the target is to find single stable compounds rather than predicting full stability vectors.